In [1]:
CONFIG = '../../logs/project/run_1/config.json'
MODEL_PATH = "../../logs/project/run_1/best.pth"

In [5]:
import json
import random

from IPython.display import Audio, display
import matplotlib.pyplot as plt
import sys
import os

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sys.path.append(os.path.abspath('../../'))

with open(CONFIG) as f:
    cfg = json.load(f)

class Config:
    def __init__(self, dictionary):
        for k, v in dictionary.items():
            if isinstance(v, dict):
                setattr(self, k, Config(v))
            else:
                setattr(self, k, v)

cfg = Config(cfg)
cfg.data.root = os.path.join('..', '..', 'data')
print(cfg.training.sampling_strategy)

{'engine': 'engine', 'early_stopping_patience': 5, 'label_smoothing': 0.0, 'batch_size': 32, 'val_freq': 1, 'epochs': 50, 'num_workers': 4, 'accum_iter': 1, 'mixed_precision': 'no', 'lr': 0.0003, 'weight_decay': 0.0001}


In [ ]:
from dataset.dataset import SpeechCommandsDataset, get_loader

train_dataset = SpeechCommandsDataset(
    root_dir=cfg.data.root,
    cfg=cfg,
    mode='training'
)

test_dataset = SpeechCommandsDataset(
    root_dir=cfg.data.root,
    cfg=cfg,
    mode='testing'
)

In [ ]:
import torch
from modeling.model import build_model

model = build_model(cfg, test_dataset.num_classes)

state_dict = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

model.eval()

print("Model loaded successfully!")

In [ ]:
def evaluate_and_display_wrong_predictions(model, test_dataset, n=5):
    model.eval() 
    
    indices = list(range(len(test_dataset)))
    random.shuffle(indices)
    
    wrong_predictions = []
    
    for idx in indices:
        data, true_label = test_dataset[idx]
        
        data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(0)
        
        with torch.no_grad():
            output = model(data_tensor)
            predicted_label = torch.argmax(output, dim=1).item()
        
        if predicted_label != true_label:
            wrong_predictions.append((idx, true_label, predicted_label))
        
        if len(wrong_predictions) >= n:
            break
    
    for idx, true_label, predicted_label in wrong_predictions:
        print(f"Sample Index: {idx}")
        print(f"True Label: {list(test_dataset.label_mapping.keys())[true_label]}, Predicted Label: {list(test_dataset.label_mapping.keys())[predicted_label]}")
        
        waveform, _ = test_dataset[idx]
        display(Audio(waveform, rate=cfg.data.sample_rate))
        print("-" * 50)

In [ ]:
evaluate_and_display_wrong_predictions(model, test_dataset, n=10)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

def evaluate_and_plot_confusion_matrix(model, test_dataset):
    model.eval()
    
    true_labels = []
    predicted_labels = []
    
    for idx in range(len(test_dataset)):
        data, true_label = test_dataset[idx]
        data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(0)
        
        with torch.no_grad():
            output = model(data_tensor)
            predicted_label = torch.argmax(output, dim=1).item()
        
        true_labels.append(true_label)
        predicted_labels.append(predicted_label)
    
    cm = confusion_matrix(true_labels, predicted_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(test_dataset.label_mapping.keys()))
    
    plt.figure(figsize=(10, 10))
    disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
    plt.title("Confusion Matrix")
    plt.show()

evaluate_and_plot_confusion_matrix(model, test_dataset)

In [ ]:
import torch.nn.functional as F

model.eval()
data, true_label = test_dataset[10]
print("True label:", true_label)
# During evaluation, after getting model outputs
outputs = model(torch.tensor(data, dtype=torch.float32).unsqueeze(0))  # Shape: [B, num_classes]

# Print raw logits
print("Logits:", outputs[0].detach().cpu().numpy())

# Convert to probabilities
probs = F.softmax(outputs, dim=1)

# Print softmax probabilities for the first sample in the batch
print("Probabilities:", probs[0].detach().cpu().numpy())

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import Subset, DataLoader

def train_on_small_subset(model, dataset, cfg, num_samples=10, num_epochs=20):
    # Create a small subset of the dataset
    subset_indices = list(range(num_samples))
    small_subset = Subset(dataset, subset_indices)
    dataloader = DataLoader(small_subset, batch_size=cfg.training.batch_size, shuffle=True)
    
    # Define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.training.lr)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    # Move model to device
    model.to(device)
    
    # Training loop
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for data, label in dataloader:
            data, label = data.to(device), label.to(device)
            
            # Forward pass
            output = model(data)
            loss = loss_fn(output, label)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Print loss for the epoch
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")
    
    print("Training on small subset complete.")

In [ ]:
model = build_model(cfg)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
train_on_small_subset(model, train_dataset, cfg, num_samples=10000, num_epochs=10)

In [ ]:
import torch.nn.functional as F

model.eval()
data, true_label = test_dataset[10]
print("True label:", true_label)
outputs = model(torch.tensor(data, dtype=torch.float32).unsqueeze(0).to(device))  # Shape: [B, num_classes]

print("Logits:", outputs[0].detach().cpu().numpy())

probs = F.softmax(outputs, dim=1)

print("Probabilities:", f"{probs[0].detach().cpu().numpy().round(2)}")

In [ ]:
model.to('cpu')
evaluate_and_plot_confusion_matrix(model, test_dataset)